# Image Segmentation

Some basic parameters:

In [ ]:
VERSION = "J040c"

# do we run on Jarvis cloud platform?
#JARVIS = True
JARVIS = False

TESTRUN = False#True

installing libraries

In [ ]:
#Run this once per session
!pip install fastai -q --upgrade
!pip install git+https://github.com/WaterKnight1998/SemTorch

# Libraries

In [ ]:
from fastai.vision.all import *
import pandas as pd
import gc # garbage collector

# SemTorch
from semtorch import get_segmentation_learner

In [ ]:
# fix randomness
my_seed = 42
np.random.seed(my_seed);random.seed(my_seed);set_seed(my_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Choose class used in this notebook

In [ ]:
# what class are we looking for?
#myclass = "aguada"
myclass = "building"
#myclass = "platform"

mymask = "mask_"+myclass

# Dataset


In [ ]:
root_dir = Path("../Data/")
root_dir

In [ ]:
path = root_dir

Some paths we use:

In [ ]:
path_im = path/'lidar_train'
# these are the original masks but 'normalized' for fastai (i.e. only 0 and 1 allowed)
path_lbl = path/'train_masks_normalized'

# this contains the pixel counts
maya_csv = path/'maya_analysis.csv'

if myclass == 'aguada': ### ????????????????????????
    # old synthetic images
    synth_path_im = path/'03_generated/images'
    synth_path_lbl = path/'03_generated/normalized_masks'
else:
    # new synthetic images (no padding) for BUILDING
    synth_path_im = path/'03_generated_21-05-20/images'
    synth_path_lbl = path/'03_generated_21-05-20/normalized_masks'
    
    # newest (rectangular cut) synthetic images
    #synth_path_im = path/'03_generated_21-06-07/images'
    #synth_path_lbl = path/'03_generated_21-06-07/normalized_masks'

    # newest synthetic images (pad 8 px)
    #synth_path_im = path/'03_generated_21-06-14_padded/images'
    #synth_path_lbl = path/'03_generated_21-06-14_padded/normalized_masks'


### Important note:

The data collection below is much more complicated that it should be. This is because we tried some data filtering first, but that did not work out. However there was no time to remove this code. We will remove it before the ECML conference, however.

In essence you will notice that we create a pandas dataframe containing all images and all masks (i.e  do *not* use the option to remove any) and add the synthetic images+masks.

(It is based on a csv file named maya_analysis.csv which contains a pixel count for each mask. But we do not use that information, as it turned out to be not helping the training if we use Focal Loss as loss function.)

First we collect our filenames. Only these with nonzero data in their mask. 

In [ ]:
mymaskDF = pd.read_csv(maya_csv)
mymaskDF['fpath'] = str(path_im) + '/' +mymaskDF['name']
print (len(mymaskDF))
mymaskDF.head()

In [ ]:
# get only these with nonzero pixCount of our class
mydf = mymaskDF[(mymaskDF['class'] == myclass) & (mymaskDF['pixCount'] > 0)]
print (len(mydf))
mydf.head()

#### Now we add the synthesized images

In [ ]:
import glob
synthDF = pd.DataFrame(glob.glob(str(synth_path_im/myclass)+'/*.*'), columns = ["name"])
synthDF['class'] = myclass
synthDF['pixCount'] = -1 # dummy marker for 'generated'
synthDF['fpath'] = synthDF['name']

In [ ]:
print(len(synthDF))
synthDF.head()

In [ ]:
# get some more from the zero pixCount images
keep = 1.  # we keep 100% so the filtering is not used as stated above :-)
otherdf = mymaskDF[(mymaskDF['class'] == myclass) & (mymaskDF['pixCount'] == 0)].sample(frac=keep)
print (len(otherdf))
otherdf.head()

In [ ]:
# and oversample "good" cases (only needed for Aguada which we oversample 6-fold)
if (myclass == "aguada"):
    mydf = mydf.sample(n=6*64, replace=True)
print (len(mydf))

In [ ]:
### MERGE HERE
mydf = mydf.append(otherdf, ignore_index=True, sort=False)
print (len(mydf))

In [ ]:
# now add the synthesized images, too
mydf = mydf.append(synthDF, ignore_index=True, sort=False)
print (len(mydf))

In [ ]:
#fnames = [path_im/f for f in mydf['name']]
fnames = [Path(f) for f in mydf['fpath']]
len(fnames)

And now let's look at the data to see if everything is ok:

In [ ]:
# tile 44 contains all three classes
img_fn = Path(root_dir/'lidar_train/tile_44_lidar.tif')
img = PILImage.create(img_fn)
img.show(figsize=(5,5))

**Now** let's grab our y's. They live in the `train_masks` folder and follow this naming pattern:

tile_(n)\_lidar.tif

tile_(n)\_mask_aguada.tif

In [ ]:
def get_msk(fn):
    # modify for synthetic images
    if str(fn)[-5] == "d": # as in "generateD", this is clumsy but we are short of time...!
        return synth_path_lbl/fn.name.replace("lidar", mymask)
    else :
        return path_lbl/fn.name.replace("lidar", mymask)

In [ ]:
# test mask path for "normal" images
print(fnames[0])
get_msk(fnames[0])

In [ ]:
# test mask path for synthetic images
print(synthDF['fpath'][0])
get_msk(Path(synthDF['fpath'][0]))

Our masks are of type `PILMask` and we will make our gradient percentage (alpha) equal to 1 as we are not overlaying this on anything yet

In [ ]:
msk = PILMask.create(get_msk(img_fn))
msk.show(figsize=(5,5), alpha=1)

We normalized the masks in advance for easier processing with fast.ai: They now contain only 0 (background) and 1 (class):

In [ ]:
np.unique(tensor(msk))

Here we name them:

In [ ]:
codes = np.array(['background', mymask])
codes

### Progressive resizing

This first round we will train at half the image size

In [ ]:
sz = msk.shape; sz

In [ ]:
half = tuple(int(x/2) for x in sz); half

In [ ]:
# batch size
bs = 8

if myclass == "platform": bs = 8
if myclass == 'building': bs = 8

if JARVIS:
    if myclass == 'aguada': bs = 80
    elif myclass == 'building': bs = 120
    else :                      bs = 100 # platform

bs

In [ ]:
def my_transforms(imgsize):
    item_tfms = [Resize(imgsize)]
    batch_tfms = [Dihedral(),Brightness(0.1,p=0.25), Zoom(max_zoom=1.1,p=0.25),
                  Normalize.from_stats(*imagenet_stats)
                 ]
    return item_tfms, batch_tfms

if myclass == 'platform': # found to be working slightly better
    def my_transforms(imgsize):
        item_tfms = None 
        batch_tfms = [Resize(imgsize), Dihedral(),Brightness(0.1,p=0.25), Zoom(max_zoom=1.1,p=0.25),
                      Normalize.from_stats(*imagenet_stats)
                     ]
        return item_tfms, batch_tfms

In [ ]:
# shuffle dataframe, just in case!
mydf = mydf.sample(frac=1).reset_index(drop=True)

In [ ]:
# mask retrieval function for dataframe rows, same as above but works with the dataframe
def get_msk(row):
    fn = Path(row["fpath"])
    # modify for synthetic images
    if str(fn)[-5] == "d": # as in "generateD", this is so ugly!!
        return synth_path_lbl/fn.name.replace("lidar", mymask)
    else :
        return path_lbl/fn.name.replace("lidar", mymask)

In [ ]:
# function for retrieving a dataloaders object for a 'fold'
def get_data(mydf, fold, n_splits, codes, bs, item_tfms, batch_tfms):
    length = int(len(mydf)/n_splits)
    start = fold*length
    
    mydf['is_valid'] = False
    mydf.loc[start:start+length,'is_valid'] = True
    # the datablock   
    dblock = DataBlock(blocks=(ImageBlock, MaskBlock(codes=codes)),
                   #splitter=RandomSplitter(valid_pct=0.2),
                   splitter=ColSplitter(), #!!! is_valid is in valid_ds
                   get_x=ColReader('fpath'),
                   get_y=get_msk, item_tfms=item_tfms, batch_tfms=batch_tfms)
    # the dataloaders
    dls = dblock.dataloaders(mydf, path='', bs=bs)
    return dls

## Model: HRnet or DeeplabV3+ with ResNet101

In [ ]:
# optimizer: we use Ranger instead of Adam
opt = ranger

In [ ]:
# we were asked for IoU as metric, which is the same as Jaccard. Besides we track Dice, but do not use it.
metrics = [Dice(), JaccardCoeff()]

### Class imbalance

The Maya dataset is heavily imbalanced with only some  0.4 % of all pixels containing mask data ("1"). We use Focal Loss to overcome the class imbalance:

In [ ]:
myloss_func=FocalLossFlat(axis=1)

In [ ]:
def segmentron_splitter(model):
    return [params(model.backbone), params(model.head)]

In [ ]:
# architecture and backbone
arch="deeplabv3+";backbone="resnet101"

arch, backbone

In [ ]:
# This is an implementation of Cutmix data augmentation (https://arxiv.org/abs/1905.04899)
# Taken from here (and modified): https://forums.fast.ai/t/implementing-cutmix-in-fastaiv2/67350/16
from torch.distributions.beta import Beta

class CutMix(Callback):
    run_after,run_valid = [Normalize],False
    def __init__(self, alpha=1.): self.distrib = Beta(tensor(alpha), tensor(alpha))
    def begin_fit(self):
        self.stack_y = getattr(self.learn.loss_func, 'y_int', False)
        if self.stack_y: self.old_lf,self.learn.loss_func = self.learn.loss_func,self.lf

    def after_fit(self):
        if self.stack_y: self.learn.loss_func = self.old_lf

    def begin_batch(self):
        W, H = self.xb[0].size(3), self.xb[0].size(2)
        
        lam = self.distrib.sample((self.y.size(0),)).squeeze().to(self.x.device)
        lam = torch.stack([lam, 1-lam], 1)
        self.lam = lam.max(1)[0]
        shuffle = torch.randperm(self.y.size(0)).to(self.x.device)
        xb1,self.yb1 = tuple(L(self.xb).itemgot(shuffle)),tuple(L(self.yb).itemgot(shuffle))
        nx_dims = len(self.x.size())

        rx = (self.distrib.sample((64,))*W).type(torch.long).to(self.x.device)
        ry = (self.distrib.sample((64,))*H).type(torch.long).to(self.x.device)
        rw = (torch.sqrt(1-self.lam)*W).to(self.x.device)
        rh = (torch.sqrt(1-self.lam)*H).to(self.x.device)

        x1 = torch.round(torch.clamp(rx-rw//2, min=0, max=W)).to(self.x.device).type(torch.long)
        x2 = torch.round(torch.clamp(rx+rw//2, min=0, max=W)).to(self.x.device).type(torch.long)
        y1 = torch.round(torch.clamp(ry-rh//2, min=0, max=H)).to(self.x.device).type(torch.long)
        y2 = torch.round(torch.clamp(ry+rh//2, min=0, max=H)).to(self.x.device).type(torch.long)
        
        for i in range(len(x1)):
            self.learn.xb[0][i, :, x1[i]:x2[i], y1[i]:y2[i]] = xb1[0][i, :, x1[i]:x2[i], y1[i]:y2[i]]
        
        self.lam = (1 - ((x2-x1)*(y2-y1))/(W*H)).type(torch.float)
        
        if not self.stack_y:
            ny_dims = len(self.y.size())
            self.learn.yb = tuple(L(self.yb1,self.yb).map_zip(torch.lerp,weight=unsqueeze(self.lam, n=ny_dims-1)))

    def lf(self, pred, *yb):
        if not self.training: return self.old_lf(pred, *yb)
        with NoneReduce(self.old_lf) as lf:
            loss = torch.lerp(lf(pred,*self.yb1), lf(pred,*yb), self.lam)
        return reduce_loss(loss, getattr(self.old_lf, 'reduction', 'mean'))


In [ ]:
# these callbacks are used during training. The first three monitor training progress, Cutmix is for augmentation.
callbacks = [SaveModelCallback(monitor='jaccard_coeff'), 
             EarlyStoppingCallback(monitor='jaccard_coeff', patience=8),
             ReduceLROnPlateau(monitor='jaccard_coeff'),
             CutMix()
            ]

In [ ]:
# we use the flat cosine annealing training loop along with fastai's default fine_tune(), so we create
# our own fine_tune_flat(), modelled after Zachary Mueller's notebook from here: 
# https://www.kaggle.com/muellerzr/cassava-fastai-starter

@patch
def fine_tune_flat(self:Learner, epochs, base_lr=4e-3, freeze_epochs=1, lr_mult=100, pct_start=0.75, 
                   first_callbacks = [], second_callbacks = [], tofp32=False,**kwargs):
    "Fine-tune applied to `fit_flat_cos`"
    self.freeze()
    try:
        self.fit_flat_cos(freeze_epochs, slice(base_lr), pct_start=0.99, cbs=first_callbacks, **kwargs)
    except: pass
    gc.collect();torch.cuda.empty_cache()
    base_lr /= 2
    self.unfreeze()
    if tofp32: self.to_fp32() # set to 32 bit
    try:
        self.fit_flat_cos(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, cbs=second_callbacks)
    except: pass
    gc.collect();torch.cuda.empty_cache()

we have two training cycles, one with half size images, one with full size images:

## half size training cycle

k-fold loop:

In [ ]:
n_folds = 5

In [ ]:
# define augmentation transforms
item_tfms, batch_tfms = my_transforms(half)

for fold in range(n_folds):
    print ("----", fold)
    dls = get_data(mydf, fold, n_folds, codes, bs, item_tfms, batch_tfms) # get the dataloaders
    # here we create the segmentation learner with SemTorch. The models are all pre-trained on ImageNet
    learn = get_segmentation_learner(dls=dls, number_classes=2, segmentation_type="Semantic Segmentation",
                                 architecture_name=arch, backbone_name=backbone,
                                 metrics=metrics,
                                 splitter=segmentron_splitter,
                                 opt_func=opt,
                                 loss_func=myloss_func).to_fp16() # we use fp16 training
    
    #set hyperparameters. This should be moved out of the loop :-)
    if myclass == 'aguada':
        lr = 1e-2
        freeze_epochs = 8#19
        epochs = 16# 6
    elif myclass == 'building':
        lr = 1e-2
        freeze_epochs = 19
        epochs = 9
    elif myclass == 'platform':
        lr =1.2e-2
        freeze_epochs = 8
        epochs = 16
    
    # here is the training cycle: we train for 'freeze_epochs' with all layers (except the last) frozen,
    # then we train for 'epochs' with all layers unfrozen 
    learn.fine_tune_flat(epochs, lr, freeze_epochs, first_callbacks=callbacks, second_callbacks=callbacks)
    learn.recorder.plot_loss()
    learn.export('models/stage-1 ' + myclass + str(fold))
    print ("--- fold complete #", fold)
    print (learn.validate())
    print ("-----------------")
    del dls, learn # free memory
 
    # early stopping
    if TESTRUN: break
        
# beat 69.3

In [ ]:
#from jarviscloud import jarviscloud
#jarviscloud.pause()

## Full size training cycle

In [ ]:
# sz is 480x480 now, so we need new transforms
item_tfms, batch_tfms = my_transforms(sz)

In [ ]:
# set hyperparameters
if myclass == "building":
    lr = 0.02
    freeze_epochs = 6
    epochs = 9
elif myclass == "aguada":
    lr = 1e-2
    freeze_epochs = 8
    epochs = 16
elif myclass == 'platform':
    lr = 1e-2
    freeze_epochs = 6
    epochs = 7

lr, freeze_epochs, epochs

In [ ]:
# reduce the batch size
if arch=="deeplabv3+":
    bs = 3 # 2 for Unet
else:
    bs = 6

if JARVIS:
    if arch=="deeplabv3+":
        bs = 12 # 2 for Unet
    else:
        if myclass == 'aguada': bs = 16
        elif myclass == 'building': bs = 16
bs

In [ ]:
# we iterate over each fold
for fold in range(n_folds):
    dls = get_data(mydf, fold, n_folds, codes, bs, item_tfms, batch_tfms) # get dataloaders
    learn = load_learner('models/stage-1 ' + myclass + str(fold)) # re-load the learner
    learn.dls = dls # and insert the new dataloaders
    learn.to_fp16() # switch into fp16 training (half precision)
    learn.loss_func = myloss_func # set loss function
    # note that we now fine tune again but 2nd cycle with full precision (tofp32)
    learn.fine_tune_flat(epochs, lr, freeze_epochs, first_callbacks=callbacks, second_callbacks=callbacks, tofp32=True)
    learn.recorder.plot_loss()
    MYMODEL = "models/maya_"+VERSION+"_"+myclass+str(fold)+".pkl"
    print ("--- fold complete #", fold)
    print (learn.validate())
    print ("-----------------")
    learn.export(MYMODEL)
    del dls, learn
    if TESTRUN: break
        
# beat 73.4

In [ ]:
#from jarviscloud import jarviscloud
#jarviscloud.pause()

# END